In [ ]:
import json
import os
from itertools import chain

import matplotlib.pyplot as plt
import pandas as pd
import yaml
from IPython.display import Image

from config import SPECS_REDUCED_ACTIVS_2000_YAML, ACPF_RESULTS_DIR, FIG_RESULTS_DIR
from drawutil import utcolors, mpl_config
from util import PyomoSolution
from model import Solution as GurobipySolution

In [ ]:
def subgraph(n, graph):
    fire = {n}
    cold = set(graph.keys()) - fire
    burn = graph[n]
    while burn:
        next_burn = set.union(*[graph[m] for m in burn]) - (fire | burn)
        fire |= burn
        cold -= burn
        burn = next_burn
    return fire

def isolated_subgraphs(graph):
    n = next(iter(graph))
    fire = {n}
    cold = set(graph.keys()) - fire
    print(len(cold))
    burn = graph[n]
    while cold:
        while burn:
            next_burn = set.union(*[graph[m] for m in burn]) - (fire | burn)
            fire |= burn
            cold -= burn
            burn = next_burn
        yield fire
        fire = {}
        if cold:
            n = next(iter(cold))
            fire = {n}
            cold -= fire
            print(len(cold))
            burn = graph[n]
    if fire:
        yield fire

In [ ]:
approaches = [
    'stochastic',
    'robust'
]

approach_names = {
    'stochastic': 'SP',
    'robust': 'RO',
}

casestudies = [
    'imelda',
    'harvey',
]

pftypes = {
    'imelda': [
        'dc',
        'lpacc',
        'lpacf',
        'qpac',
    ],
    'harvey': [
        'dc',
        'lpacc',
    ]
}

f_min_max_inc = {
    ('imelda', 'stochastic'): (0,  20, 1),
    ('imelda', 'robust'):     (0,   9, 1),
    ('harvey', 'stochastic'): (0, 193, 1),
    ('harvey', 'robust'):     (0,  62, 1)
}

r_hat = 3

clrs = {
    'dc': 'blue',
    'lpacc': 'purple',
    'lpacf': 'red',
    'qpac': 'green'
}

aliases = {
    'dc': 'DC',
    'lpacc': 'LPAC\!\!-\!\!C',
    'lpacf': 'LPAC\!\!-\!\!F',
    'qpac': 'QPAC',
}

agg_func = {
    'stochastic': 'mean',
    'robust': 'max'
}

In [ ]:
with open(SPECS_REDUCED_ACTIVS_2000_YAML) as fh:
    specs = yaml.load(fh, Loader=yaml.Loader)
    n_d = dict()
    for n, ds in specs['D_n'].items():
        for d in ds:
            n_d[d] = n
    

hash_data = dict()
for casestudy in casestudies:
    for approach in approaches:
        f_min, f_max, f_inc = f_min_max_inc[casestudy, approach]
        for pftype in pftypes[casestudy]:
            for f in range(f_min, f_max + f_inc, f_inc):
                filename = f'{approach}-{casestudy}-{pftype}-f{f}-r{r_hat}.json'
                with open(os.path.join(ACPF_RESULTS_DIR, 'results', filename)) as fh:
                    hash_data[approach, casestudy, pftype, f, r_hat] = json.load(fh)
df_res = pd.DataFrame(hash_data).T.stack().to_frame()
df_res.index.names = ['approach', 'casestudy', 'pftype', 'f', 'rhat', 'omega']
df_res.columns = ['hash']


sur_data = {pftype: dict() for pftype in sorted(set(chain.from_iterable(pftypes.values())))}
val_data = {pftype: dict() for pftype in sorted(set(chain.from_iterable(pftypes.values())))}
for pftype in sorted(set(chain.from_iterable(pftypes.values()))):
    for filename in os.listdir(os.path.join(ACPF_RESULTS_DIR, pftype)):
        if '-ac' in filename:
            val_data[pftype][filename.split('-')[0]] = PyomoSolution.from_zip(os.path.join(ACPF_RESULTS_DIR, pftype, filename))
        else:
            sur_data[pftype][filename.split('-')[0]] = GurobipySolution.from_zip(os.path.join(ACPF_RESULTS_DIR, pftype, filename))


def get_sur_perf(pftype, hash):
    sol = sur_data[pftype][hash]
    return sum((1 - sol['z'][d]) * specs['p_load_hi'][d] for d in specs['D']) / 10


def get_val_perf(pftype, hash):
    sol = val_data[pftype][hash]
    return sol['p_load_shed'].sum() / 10


def get_bus_outages(pftype, hash):
    sol = sur_data[pftype][hash]
    return (1 - sol['alpha']).sum()


df_res.reset_index(inplace=True)
df_res['perf_sur'] = df_res[['pftype', 'hash']].apply(lambda x: get_sur_perf(*x), axis=1)
df_res['perf_val'] = df_res[['pftype', 'hash']].apply(lambda x: get_val_perf(*x), axis=1)
df_res['bus_outages'] = df_res[['pftype', 'hash']].apply(lambda x: get_bus_outages(*x), axis=1)

In [ ]:
SCALE = 1.0
mpl_config(SCALE)
pftype = 'dc'

# subplot sizing overhead
plot_num_w = 2
plot_num_h = len(pftypes[casestudy])
lo_pad_w, md_pad_w, hi_pad_w, plot_dim_w = 0.750 * SCALE, 1.000 * SCALE, 0.250 * SCALE, 2.500 * SCALE
lo_pad_h, md_pad_h, hi_pad_h, plot_dim_h = 0.500 * SCALE, 0.250 * SCALE, 0.250 * SCALE, 1.500 * SCALE
fig_w = lo_pad_w + hi_pad_w + md_pad_w * (plot_num_w - 1) + plot_dim_w * plot_num_w
fig_h = lo_pad_h + hi_pad_h + md_pad_h * (plot_num_h - 1) + plot_dim_h * plot_num_h
fig, axes = plt.subplots(plot_num_h, plot_num_w,
                         figsize=(fig_w, fig_h),
                         sharex='col',
                         sharey=False)
plt.subplots_adjust(left=lo_pad_w / fig_w,
                    bottom=lo_pad_h / fig_h,
                    right=(fig_w - hi_pad_w) / fig_w,
                    top=(fig_h - hi_pad_h) / fig_h,
                    wspace=md_pad_w / plot_dim_w,
                    hspace=md_pad_h / plot_dim_h)

for i, approach in enumerate(approaches):
    df_tmp = df_res.loc[df_res['approach'] == approach]\
                   .groupby(['casestudy', 'pftype', 'f'])\
                   .agg({'perf_sur': agg_func[approach], 'perf_val': agg_func[approach]})
    for j, casestudy in enumerate(casestudies):
        ax = axes[i, j]
        f_min, f_max, f_inc = f_min_max_inc[casestudy, 'stochastic']
        sr_sur = df_tmp.loc[(casestudy, pftype), 'perf_sur']\
                       .reindex(range(f_min, f_max + f_inc, f_inc))\
                       .ffill()
        sr_val = df_tmp.loc[(casestudy, pftype), 'perf_val']\
                       .reindex(range(f_min, f_max + f_inc, f_inc))\
                       .ffill()
        sr_sur.plot(ax=ax, label=('DC', approach_names[approach]), color=utcolors[3])
        sr_val.plot(ax=ax, label=('AC', approach_names[approach]), color=utcolors[1])
        legend = ax.legend(loc='upper right')
        legend.get_frame().set_edgecolor('k')
        ax.set_xlabel('$f$')
        ax.set_ylabel(agg_func[approach].title() + ' Load Shed (GW)')

for j, casestudy in enumerate(casestudies):
    axes[0, j].set_title(casestudy.split('-')[0].title())

image_jpg = os.path.join(FIG_RESULTS_DIR, f'ijoc-acpf-perf.jpg')
image_eps = os.path.join(FIG_RESULTS_DIR, f'ijoc-acpf-perf.eps')
plt.savefig(image_jpg, format='jpg', dpi=256)
plt.savefig(image_eps, format='eps')
plt.close()
display(Image(image_jpg))

In [ ]:
SCALE = 1.0
mpl_config(SCALE)
pftype = 'dc'

# subplot sizing overhead
plot_num_w = 2
plot_num_h = 1
lo_pad_w, md_pad_w, hi_pad_w, plot_dim_w = 0.750 * SCALE, 1.000 * SCALE, 0.250 * SCALE, 2.500 * SCALE
lo_pad_h, md_pad_h, hi_pad_h, plot_dim_h = 0.500 * SCALE, 0.250 * SCALE, 0.250 * SCALE, 1.500 * SCALE
fig_w = lo_pad_w + hi_pad_w + md_pad_w * (plot_num_w - 1) + plot_dim_w * plot_num_w
fig_h = lo_pad_h + hi_pad_h + md_pad_h * (plot_num_h - 1) + plot_dim_h * plot_num_h
fig, axes = plt.subplots(plot_num_h, plot_num_w,
                         figsize=(fig_w, fig_h),
                         sharex='col',
                         sharey=False)
plt.subplots_adjust(left=lo_pad_w / fig_w,
                    bottom=lo_pad_h / fig_h,
                    right=(fig_w - hi_pad_w) / fig_w,
                    top=(fig_h - hi_pad_h) / fig_h,
                    wspace=md_pad_w / plot_dim_w,
                    hspace=md_pad_h / plot_dim_h)


df_tmp = df_res.loc[df_res['pftype'] == 'dc']
axes[0].plot(df_tmp['bus_outages'], df_tmp['perf_val'] - df_tmp['perf_sur'],
             marker='o', markersize=1, linestyle='none', color='k')
axes[0].set_ylabel('Absolute Error in\nLoad Shed (GW)')
axes[0].set_xlabel('\# Bus Outages')
axes[1].plot(df_tmp['bus_outages'], 100 * (df_tmp['perf_val'] - df_tmp['perf_sur']) / df_tmp['perf_val'],
             marker='o', markersize=1, linestyle='none', color='k')
axes[1].set_ylabel('Relative Error in\nLoad Shed (\%)')
axes[1].set_xlabel('\# Bus Outages')

image_jpg = os.path.join(FIG_RESULTS_DIR, f'ijoc-acpf-error.jpg')
image_eps = os.path.join(FIG_RESULTS_DIR, f'ijoc-acpf-error.eps')
plt.savefig(image_jpg, format='jpg', dpi=256)
plt.savefig(image_eps, format='eps')
plt.close()
display(Image(image_jpg))